In [ ]:
import os
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from sb3_contrib import MaskablePPO
from stable_baselines3.common.policies import ActorCriticPolicy
import gymnasium as gym
from gymnasium import spaces
from datetime import datetime
import glob
import torchOptics.optics as tt
import torch.nn as nn
import torchOptics.metrics as tm
import torch.nn.functional as F
import torch.optim
import torch
from torch.utils.data import Dataset, DataLoader
import os
import glob
import matplotlib.pyplot as plt
import pickle
import torchvision
import tqdm
import time
import pandas as pd
from sb3_contrib.common.maskable.utils import get_action_masks
from sb3_contrib.common.wrappers import ActionMasker
from stable_baselines3 import PPO
import warnings

warnings.filterwarnings('ignore')

# 현재 날짜와 시간을 가져와 포맷 지정
current_date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

torch.backends.cudnn.enabled = False

class BinaryNet(nn.Module):
    def __init__(self, num_hologram, final='Sigmoid', in_planes=3,
                 channels=[32, 64, 128, 256, 512, 1024, 2048, 4096],
                 convReLU=True, convBN=True, poolReLU=True, poolBN=True,
                 deconvReLU=True, deconvBN=True):
        super(BinaryNet, self).__init__()

        def CRB2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True, relu=True, bn=True):
            layers = []
            layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                 kernel_size=kernel_size, stride=stride, padding=padding,
                                 bias=bias)]
            if relu:
                layers += [nn.Tanh()]
            if bn:
                layers += [nn.BatchNorm2d(num_features=out_channels)]

            cbr = nn.Sequential(*layers)  # *으로 list unpacking

            return cbr

        def TRB2d(in_channels, out_channels, kernel_size=2, stride=2, bias=True, relu=True, bn=True):
            layers = []
            layers += [nn.ConvTranspose2d(in_channels=in_channels, out_channels=out_channels,
                                          kernel_size=2, stride=2, padding=0,
                                          bias=True)]
            if bn:
                layers += [nn.BatchNorm2d(num_features=out_channels)]
            if relu:
                layers += [nn.ReLU()]

            cbr = nn.Sequential(*layers)  # *으로 list unpacking

            return cbr

        self.enc1_1 = CRB2d(in_planes, channels[0], relu=convReLU, bn=convBN)
        self.enc1_2 = CRB2d(channels[0], channels[0], relu=convReLU, bn=convBN)
        self.pool1 = CRB2d(channels[0], channels[0], stride=2, relu=poolReLU, bn=poolBN)

        self.enc2_1 = CRB2d(channels[0], channels[1], relu=convReLU, bn=convBN)
        self.enc2_2 = CRB2d(channels[1], channels[1], relu=convReLU, bn=convBN)
        self.pool2 = CRB2d(channels[1], channels[1], stride=2, relu=poolReLU, bn=poolBN)

        self.enc3_1 = CRB2d(channels[1], channels[2], relu=convReLU, bn=convBN)
        self.enc3_2 = CRB2d(channels[2], channels[2], relu=convReLU, bn=convBN)
        self.pool3 = CRB2d(channels[2], channels[2], stride=2, relu=poolReLU, bn=poolBN)

        self.enc4_1 = CRB2d(channels[2], channels[3], relu=convReLU, bn=convBN)
        self.enc4_2 = CRB2d(channels[3], channels[3], relu=convReLU, bn=convBN)
        self.pool4 = CRB2d(channels[3], channels[3], stride=2, relu=poolReLU, bn=poolBN)

        self.enc5_1 = CRB2d(channels[3], channels[4], relu=convReLU, bn=convBN)
        self.enc5_2 = CRB2d(channels[4], channels[4], relu=convReLU, bn=convBN)

        self.deconv4 = TRB2d(channels[4], channels[3], relu=deconvReLU, bn=deconvBN, stride=2)
        self.dec4_1 = CRB2d(channels[4], channels[3], relu=convReLU, bn=convBN)
        self.dec4_2 = CRB2d(channels[3], channels[3], relu=convReLU, bn=convBN)

        self.deconv3 = TRB2d(channels[3], channels[2], relu=deconvReLU, bn=deconvBN, stride=2)
        self.dec3_1 = CRB2d(channels[3], channels[2], relu=convReLU, bn=convBN)
        self.dec3_2 = CRB2d(channels[2], channels[2], relu=convReLU, bn=convBN)

        self.deconv2 = TRB2d(channels[2], channels[1], relu=deconvReLU, bn=deconvBN, stride=2)
        self.dec2_1 = CRB2d(channels[2], channels[1], relu=convReLU, bn=convBN)
        self.dec2_2 = CRB2d(channels[1], channels[1], relu=convReLU, bn=convBN)

        self.deconv1 = TRB2d(channels[1], channels[0], relu=deconvReLU, bn=deconvBN, stride=2)
        self.dec1_1 = CRB2d(channels[1], channels[0], relu=convReLU, bn=convBN)
        self.dec1_2 = CRB2d(channels[0], channels[0], relu=convReLU, bn=convBN)

        self.classifier = CRB2d(channels[0], num_hologram, relu=False, bn=False)

    def forward(self, x):
        # Encoder
        enc1_1 = self.enc1_1(x)
        enc1_2 = self.enc1_2(enc1_1)
        pool1 = self.pool1(enc1_2)

        enc2_1 = self.enc2_1(pool1)
        enc2_2 = self.enc2_2(enc2_1)
        pool2 = self.pool2(enc2_2)

        enc3_1 = self.enc3_1(pool2)
        enc3_2 = self.enc3_2(enc3_1)
        pool3 = self.pool3(enc3_2)

        enc4_1 = self.enc4_1(pool3)
        enc4_2 = self.enc4_2(enc4_1)
        pool4 = self.pool4(enc4_2)

        enc5_1 = self.enc5_1(pool4)
        enc5_2 = self.enc5_2(enc5_1)

        deconv4 = self.deconv4(enc5_2)
        concat4 = torch.cat((deconv4, enc4_2), dim=1)
        dec4_1 = self.dec4_1(concat4)
        dec4_2 = self.dec4_2(dec4_1)

        deconv3 = self.deconv3(dec4_2)
        concat3 = torch.cat((deconv3, enc3_2), dim=1)
        dec3_1 = self.dec3_1(concat3)
        dec3_2 = self.dec3_2(dec3_1)

        deconv2 = self.deconv2(dec3_2)
        concat2 = torch.cat((deconv2, enc2_2), dim=1)
        dec2_1 = self.dec2_1(concat2)
        dec2_2 = self.dec2_2(dec2_1)

        deconv1 = self.deconv1(dec2_2)
        concat1 = torch.cat((deconv1, enc1_2), dim=1)
        dec1_1 = self.dec1_1(concat1)
        dec1_2 = self.dec1_2(dec1_1)

        # Final classifier
        out = self.classifier(dec1_2)
        out = nn.Sigmoid()(out)
        return out


model = BinaryNet(num_hologram=8, in_planes=1, convReLU=False,
                  convBN=False, poolReLU=False, poolBN=False,
                  deconvReLU=False, deconvBN=False).cuda()
test = torch.randn(1, 1, 1024, 1024).cuda()
out = model(test)
print(out.shape)


class Dataset512(Dataset):
    def __init__(self, target_dir, meta, transform=None, isTrain=True, padding=0):
        self.target_dir = target_dir
        self.transform = transform
        self.meta = meta
        self.isTrain = isTrain
        self.target_list = sorted(glob.glob(target_dir+'*.png'))
        self.center_crop = torchvision.transforms.CenterCrop(1024)
        self.random_crop = torchvision.transforms.RandomCrop((1024, 1024))
        self.padding = padding

    def __len__(self):
        return len(self.target_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        target = tt.imread(self.target_list[idx], meta=meta, gray=True).unsqueeze(0)
        if target.shape[-1] < 1024 or target.shape[-2] < 1024:
            target = torchvision.transforms.Resize(1024)(target)
        if self.isTrain:
            target = self.random_crop(target)
            target = torchvision.transforms.functional.pad(target, (self.padding, self.padding, self.padding, self.padding))
        else:
            target = self.center_crop(target)
            target = torchvision.transforms.functional.pad(target, (self.padding, self.padding, self.padding, self.padding))
        return target


#BinaryHologramEnv 클래스
class BinaryHologramEnv(gym.Env):
    def __init__(self, target_function, trainloader, max_steps=100000, T_PSNR=30, T_steps=1000):
        """
        target_function: 타겟 이미지와의 손실(MSE 또는 PSNR) 계산 함수.
        trainloader: 학습 데이터셋 로더.
        max_steps: 최대 타임스텝 제한.
        T_PSNR: 목표 PSNR 값.
        T_steps: PSNR 목표를 유지해야 하는 최소 타임스텝.
        """
        super(BinaryHologramEnv, self).__init__()

        # 관찰 공간 (1, 8, 1024, 1024)
        self.observation_space = spaces.Box(low=0, high=1, shape=(1, 8, 1024, 1024), dtype=np.float32)

        # 행동 공간: MultiBinary 데이터
        self.action_space = spaces.MultiBinary(1 * 8 * 1024 * 1024)

        # 모델 및 데이터 로더 설정
        self.target_function = target_function  # BinaryNet 모델
        self.trainloader = trainloader          # 학습 데이터 로더

        # 에피소드 설정
        self.max_steps = max_steps
        self.T_PSNR = T_PSNR
        self.T_steps = T_steps

        # 학습 상태
        self.state = None
        self.observation = None
        self.steps = 0
        self.psnr_sustained_steps = 0

        # 학습 데이터셋에서 첫 배치 추출
        self.data_iter = iter(self.trainloader)
        self.target_image = None

    def reset(self, seed=None, options=None, lr=1e-4, z=2e-3):
        torch.cuda.empty_cache()
        try:
            self.target_image = next(self.data_iter)
        except StopIteration:
            self.data_iter = iter(self.trainloader)
            self.target_image = next(self.data_iter)
    
        self.target_image = self.target_image.cuda()
        with torch.no_grad():
            model_output = self.target_function(self.target_image)
        self.observation = model_output.cpu().numpy()  # (1, 8, 1024, 1024)

        self.steps = 0
        self.psnr_sustained_steps = 0
        self.state = (self.observation >= 0.5).astype(np.int8)  # 이진화 상태

        binary = torch.tensor(self.state, dtype=torch.float32).cuda()
        binary = tt.Tensor(binary, meta={'dx': (7.56e-6, 7.56e-6), 'wl': 515e-9})  # meta 정보 포함

        # 시뮬레이션
        sim = tt.simulate(binary, z).abs()**2
        result = torch.mean(sim, dim=1, keepdim=True)

        # MSE 및 PSNR 계산
        mse = tt.relativeLoss(result, self.target_image, F.mse_loss).detach().cpu().numpy()
        psnr = tt.relativeLoss(result, self.target_image, tm.get_PSNR)

        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"Initial MSE: {mse:.6f}, Initial PSNR: {psnr:.6f}, {current_time}")

        mask = self.create_action_mask(self.observation)
        return self.observation, {"state": self.state, "mask": mask}


    def initialize_state(self, z=2e-3):
        """
        초기 상태를 생성하고, 시뮬레이션 및 관련 값을 계산합니다.
        """
        with torch.no_grad():
            model_output = self.target_function(self.target_image)
        self.observation = model_output.cpu().numpy()  # (1, 8, 1024, 1024)

        self.state = (self.observation >= 0.5).astype(np.int8)  # 이진화 상태

        binary = torch.tensor(self.state, dtype=torch.float32).cuda()
        binary = tt.Tensor(binary, meta={'dx': (7.56e-6, 7.56e-6), 'wl': 515e-9})  # meta 정보 포함

        # 시뮬레이션
        sim = tt.simulate(binary, z).abs()**2
        result = torch.mean(sim, dim=1, keepdim=True)

        # MSE 및 PSNR 계산
        mse = tt.relativeLoss(result, self.target_image, F.mse_loss).detach().cpu().numpy()
        psnr = tt.relativeLoss(result, self.target_image, tm.get_PSNR)

        print(f"Initial MSE: {mse:.6f}, Initial PSNR: {psnr:.6f}, {current_date}")

        # 관찰값 업데이트
        self.observation = result.detach().cpu().numpy()
        mask = self.create_action_mask(self.observation)

        return self.observation, {"state": self.state, "mask": mask}

    def step(self, action, lr=1e-4, z=2e-3):
        # 첫 스텝에서 초기 상태와 동일한 행동 적용
        if self.steps == 0:
            print("Executing reset logic for the first step")
            self.steps += 1  # 스텝 증가
            # reset과 동일한 로직을 호출해 초기 상태 생성
            observation, info = self.initialize_state(z)
            return observation, 0.0, False, False, info

        # 행동(action)을 이진 상태(state)와 동일한 형식으로 변환
        action = np.reshape(action, (1, 8, 1024, 1024)).astype(np.int8)

        # 현재 상태에 행동을 적용하여 새로운 상태 생성
        new_state = np.logical_xor(self.state, action).astype(np.float32)

        # 이진화된 새로운 상태를 torch 텐서로 변환
        binary = torch.tensor(new_state, dtype=torch.float32).cuda()
        binary = tt.Tensor(binary, meta={'dx': (7.56e-6, 7.56e-6), 'wl': 515e-9})  # meta 정보 포함

        # 시뮬레이션 수행
        sim = tt.simulate(binary, z).abs()**2
        result = torch.mean(sim, dim=1, keepdim=True)

        # MSE 및 PSNR 계산
        mse = tt.relativeLoss(result, self.target_image, F.mse_loss).detach().cpu().numpy()
        psnr = tt.relativeLoss(result, self.target_image, tm.get_PSNR)
        reward = -mse

        # 출력 추가
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"Step: {self.steps}, MSE: {mse:.6f}, PSNR: {psnr:.6f}, {current_time}")

        # 상태 업데이트
        self.state = new_state
        self.observation = result.detach().cpu().numpy()

        # 종료 조건
        terminated = self.steps >= self.max_steps or self.psnr_sustained_steps >= self.T_steps
        truncated = self.steps >= self.max_steps

        if psnr >= self.T_PSNR:
            self.psnr_sustained_steps += 1
        else:
            self.psnr_sustained_steps = 0

        # 행동 마스크 생성
        mask = self.create_action_mask(self.observation)
        info = {"mse": mse, "psnr": psnr, "mask": mask}

        del binary, sim, result
        torch.cuda.empty_cache()

        self.steps += 1
        return self.observation, reward, terminated, truncated, info


    def create_action_mask(self, observation):
        """
        관찰값에 따라 행동 마스크 생성.
        관찰값이 0~0.2인 경우 -> 행동 0으로 고정.
        관찰값이 0.8~1인 경우 -> 행동 1로 고정.
        """
        mask = np.ones_like(observation, dtype=np.int8)  # 기본적으로 모든 행동 가능
        mask[observation <= 0.49] = 0  # 관찰값이 0~0.2면 행동 0으로 고정
        mask[observation >= 0.51] = 1  # 관찰값이 0.8~1이면 행동 1로 고정
        return mask


def initialize_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_uniform_(m.weight.data, nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight.data, 1)
        nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight.data)
        nn.init.constant_(m.bias.data, 0)

batch_size = 1
target_dir = '/nfs/dataset/DIV2K/DIV2K_train_HR/DIV2K_train_HR/'
valid_dir = '/nfs/dataset/DIV2K/DIV2K_valid_HR/DIV2K_valid_HR/'
meta = {'wl': (515e-9), 'dx': (7.56e-6, 7.56e-6)}  # 메타 정보
padding = 0

# Dataset512 클래스 사용
train_dataset = Dataset512(target_dir=target_dir, meta=meta, isTrain=True, padding=padding)
valid_dataset = Dataset512(target_dir=valid_dir, meta=meta, isTrain=False, padding=padding)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# BinaryNet 모델 로드
model = BinaryNet(num_hologram=8, in_planes=1, convReLU=False, convBN=False,
                  poolReLU=False, poolBN=False, deconvReLU=False, deconvBN=False).cuda()
model.load_state_dict(torch.load('result_v/2024-12-15 14:02:27.770108_pre_reinforce_8_0.002/2024-12-15 14:02:27.770108_pre_reinforce_8_0.002'))
model.eval()


# 마스크 함수 정의
def mask_fn(env):
    return env.create_action_mask(env.observation)

# 환경 생성에 새로운 데이터 로더 적용
env = BinaryHologramEnv(
    target_function=model,
    trainloader=train_loader,  # 업데이트된 train_loader 사용
    max_steps=100000,
    T_PSNR=30,
    T_steps=1000
)

# ActionMasker 래퍼 적용
env = ActionMasker(env, mask_fn)

# Vectorized 환경 생성
venv = make_vec_env(lambda: env, n_envs=1)
venv = VecNormalize(venv, norm_obs=True, norm_reward=True, clip_obs=10.0)

# PPO 학습
ppo_model = PPO(
    "MlpPolicy",
    venv,
    verbose=2,
    n_steps=1024,
    batch_size=64,
    gamma=0.99,
    learning_rate=3e-4,
    tensorboard_log="./ppo_with_mask/"
)

ppo_model.learn(total_timesteps=10000000)

# 학습된 모델 저장
ppo_model.save(f"ppo_with_mask_{current_date}")


# 평가용 환경 생성
#eval_env = make_vec_env(lambda: env, n_envs=1)

# EvalCallback 추가
#eval_callback = EvalCallback(
#    eval_env,
#    best_model_save_path='./logs/',
#    log_path='./logs/',
#    eval_freq=10000,  # 평가 빈도 (타임스텝 기준)
#    deterministic=True,
#    render=False
#)

#ppo_model = PPO(
#    "MlpPolicy",
#    venv,
#    verbose=2,
#    n_steps=1024,
#    batch_size=64,
#    gamma=0.99,
#    learning_rate=3e-4,
#    tensorboard_log="./ppo_with_mask/"
#)

# 학습 시작 (콜백 추가)
#ppo_model.learn(total_timesteps=10000000, callback=eval_callback)

/usr/local/lib/python3.8/dist-packages/kornia/feature/lightglue.py:30: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


torch.Size([1, 8, 1024, 1024])
Using cuda device
Initial MSE: 0.002248, Initial PSNR: 26.481445, 09:39:51
Logging to ./ppo_with_mask/PPO_46
Executing reset logic for the first step
Initial MSE: 0.002248, Initial PSNR: 26.481445, 2024-12-20_09-39-10
Step: 1, MSE: 0.063607, PSNR: 11.964958, 09:39:52
Step: 2, MSE: 0.063748, PSNR: 11.955311, 09:39:52
Step: 3, MSE: 0.063590, PSNR: 11.966084, 09:39:52
Step: 4, MSE: 0.063558, PSNR: 11.968312, 09:39:53
Step: 5, MSE: 0.063581, PSNR: 11.966709, 09:39:53
Step: 6, MSE: 0.063746, PSNR: 11.955450, 09:39:53
Step: 7, MSE: 0.063700, PSNR: 11.958619, 09:39:53
Step: 8, MSE: 0.063676, PSNR: 11.960274, 09:39:54
Step: 9, MSE: 0.063705, PSNR: 11.958247, 09:39:54
Step: 10, MSE: 0.063620, PSNR: 11.964094, 09:39:54
Step: 11, MSE: 0.063649, PSNR: 11.962078, 09:39:55
Step: 12, MSE: 0.063637, PSNR: 11.962875, 09:39:55
Step: 13, MSE: 0.063630, PSNR: 11.963376, 09:39:55
Step: 14, MSE: 0.063702, PSNR: 11.958437, 09:39:55
Step: 15, MSE: 0.063763, PSNR: 11.954308, 09:3